In [39]:
df=spark.createDataFrame([['test'],['malayalam'],['check'],['malaYALAM']],['words'])

In [40]:
df.show()

+---------+
|    words|
+---------+
|     test|
|malayalam|
|    check|
|malaYALAM|
+---------+



In [199]:
import pyspark.sql.functions as f

In [8]:
import numpy as np

In [150]:
def palindrome(row):
    return True if row[::-1].lower()==row.lower() else False

In [46]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType


In [47]:
check_palindrome=udf(palindrome,BooleanType())

In [49]:
df2=df.select('*',check_palindrome('words'))

In [50]:
df2.show()

+---------+-----------------+
|    words|palindrome(words)|
+---------+-----------------+
|     test|            false|
|malayalam|             true|
|    check|            false|
|malaYALAM|             true|
+---------+-----------------+



In [51]:
df2.dtypes

[('words', 'string'), ('palindrome(words)', 'boolean')]

In [52]:
df_test=spark.range(10)

In [58]:
df_test.select(f.col('id')*f.col('id')).show()

+---------+
|(id * id)|
+---------+
|        0|
|        1|
|        4|
|        9|
|       16|
|       25|
|       36|
|       49|
|       64|
|       81|
+---------+



In [62]:
import pyarrow

In [61]:
pyarrow.__version__

'0.14.1'

In [85]:
spark.conf.get("spark.sql.execution.arrow.enabled", "true")

u'true'

In [28]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType
from pyspark.sql.types import *

In [154]:
def vector_palindrome(pandas_series):
    return pandas_series.apply(palindrome)

In [159]:
vectorized_palindrome=pandas_udf(vector_palindrome,returnType=BooleanType(),functionType=PandasUDFType.SCALAR)

In [161]:
df.select('*',vectorized_palindrome('words')).show()

+---------+------------------------+
|    words|vector_palindrome(words)|
+---------+------------------------+
|     test|                   false|
|malayalam|                    true|
|    check|                   false|
|malaYALAM|                    true|
+---------+------------------------+



In [25]:
df_purchase=spark.createDataFrame([(1,3),(2,4),(3,5),(1,5),(2,7),(3,5)],['id','purchase'])

In [26]:
df_purchase.show()

+---+--------+
| id|purchase|
+---+--------+
|  1|       3|
|  2|       4|
|  3|       5|
|  1|       5|
|  2|       7|
|  3|       5|
+---+--------+



In [142]:
from scipy.stats.mstats import gmean
def count_udf(df):
    #print(df)
    df['geometric_mean']=gmean(df.purchase)
    return df

In [141]:
vectorized_geometric=\
pandas_udf(count_udf,returnType="id bigint,purchase bigint,geometric_mean double",\
           functionType=PandasUDFType.GROUPED_MAP)

In [139]:
df_opt=df_purchase.groupby('id').apply(vectorized_geometric)

In [144]:
df_opt.show()

+---+--------+------------------+
| id|purchase|    geometric_mean|
+---+--------+------------------+
|  1|       3| 3.872983346207417|
|  1|       5| 3.872983346207417|
|  3|       5| 4.999999999999999|
|  3|       5| 4.999999999999999|
|  2|       4|5.2915026221291805|
|  2|       7|5.2915026221291805|
+---+--------+------------------+



In [140]:
df_opt.dtypes

[('id', 'bigint'), ('purchase', 'bigint'), ('geometric_mean', 'double')]

In [227]:
@pandas_udf("int", PandasUDFType.GROUPED_AGG)
def odd_sum(df):
    return (df[df %2 ==1 ].count())

In [228]:
df_purchase.groupBy('id').agg(odd_sum(df_purchase['purchase'])).show()

+---+-----------------+
| id|odd_sum(purchase)|
+---+-----------------+
|  1|                2|
|  3|                2|
|  2|                1|
+---+-----------------+

